# 1. Data Processing

In [1]:
# import the libraries 
import numpy as np
import pandas as pd 
from ortools.sat.python import cp_model

In [4]:
# Read Data 
def input(file):
    with open(file, 'r') as file:
        # read first line 
        N, D, a, b = [int(x) for x in file.readline().split()] 

        # Matrix (n,d) full 0, if staff i rest day d(i) -> convert to 1 
        F = np.full((N, D), 0)  
        for staff in range(N):
            # read each line to end, [:-1] bcs end of each line is -1 
            temp = [int(x) for x in file.readline().split()[:-1]]  #check each line from 2 -> i+1
            for day in temp:
                F[staff, day-1] = 1 # day-1 bcs index of list
    return N, D, a, b, F

# Input 
N, D, a, b, F = input("data.txt")

In [5]:
print("Number of Staffs: {}\nNumber of Days: {}\nRange staffs of a shift: {} -> {}\nList rest day of staff i:\n{}".format(N,D,a,b,F))

Number of Staffs: 9
Number of Days: 5
Range staffs of a shift: 1 -> 3
List rest day of staff i:
[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]


In [6]:
# Check status to apply color for each type of status
def status_color(value):
  if value == "Rest": 
    color = 'Green'
  elif value == "Nigh":
    color = 'Red'
  else:
    color = 'White'
  return 'background-color: %s' % color

# 2. CSOP

## 2.1. Optimization

In [7]:
# Declare the Model 
model = cp_model.CpModel()

# Create the Variables
# X[staff, day, shift] = 1 if staff i work on shift k of day j 
# X[staff, day, shift] = 0, otw
X = {} 
for staff in range(N):              # check each staff 
    for day in range(D):            # check each staff
        for shift in range(1,5):    # check each shift
            X[staff, day, shift] = model.NewIntVar(0,1,"X[{},{},{}]".format(staff,day,shift))

In [8]:
# Each day, a staff can only work one shift at most
for staff in range(N):    
    for day in range(D):   
        if F[staff, day] == 0:
            if day == 0:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 1)
            # If you work the night shift the day before, you can rest the next day
            else:
                model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) + X[staff, day - 1, 4] == 1)
        else: # F[staff, day] == 1
            model.Add(sum([X[staff, day, shift] for shift in range(1,5)]) == 0)

In [9]:
# Each shift in each day has at least [a] staffs and at most [b] staffs
for day in range(D):               
    for shift in range(1,5):    
        model.Add(sum([X[staff, day, shift] for staff in range(N)]) >= a)
        model.Add(sum([X[staff, day, shift] for staff in range(N)]) <= b)

In [11]:
# F(i): list of staff rest days i
# The maximum number of night shifts assigned to a specific staff is the smallest

max_night_shift = model.NewIntVar(1, int(D/2) + 1, 'max_night_shift') # limit rest day = 1/2 all days
# for loop add constraint confirm sum of all night shift of staff <= max_night_shift
for staff in range(N):
    model.Add(sum([X[staff, day, 4] for day in range(D)]) - max_night_shift <= 0)

In [23]:
# Objective Function
model.Minimize(max_night_shift)

# Solver
solver = cp_model.CpSolver()
status = solver.Solve(model)
if __name__ == '__main__':
  if status == cp_model.OPTIMAL:
    print("Optimal Value:", int(solver.ObjectiveValue()))
    for staff in range(N):
        for day in range(D):
            for shift in range(1, 5):
              if int(solver.Value(X[staff, day, shift])) == 1:
                print(f'Staff {staff+1} works on day {day+1}, at shift {shift}')
  else: 
    print("No Optimal Solution!")

Optimal Value: 1
Staff 1 works on day 1, at shift 1
Staff 1 works on day 2, at shift 4
Staff 1 works on day 4, at shift 3
Staff 1 works on day 5, at shift 2
Staff 2 works on day 1, at shift 2
Staff 2 works on day 2, at shift 2
Staff 2 works on day 3, at shift 1
Staff 2 works on day 5, at shift 4
Staff 3 works on day 1, at shift 3
Staff 3 works on day 2, at shift 1
Staff 3 works on day 3, at shift 2
Staff 3 works on day 4, at shift 1
Staff 3 works on day 5, at shift 2
Staff 4 works on day 1, at shift 1
Staff 4 works on day 2, at shift 1
Staff 4 works on day 3, at shift 1
Staff 4 works on day 4, at shift 4
Staff 5 works on day 1, at shift 1
Staff 5 works on day 2, at shift 3
Staff 5 works on day 3, at shift 4
Staff 5 works on day 5, at shift 2
Staff 6 works on day 1, at shift 3
Staff 6 works on day 2, at shift 3
Staff 6 works on day 3, at shift 4
Staff 6 works on day 5, at shift 3
Staff 7 works on day 1, at shift 4
Staff 7 works on day 3, at shift 1
Staff 7 works on day 4, at shift 2
Sta

## 2.2. Visualization

In [29]:
# Matrix (n,d,s=5) full 0, if staff i works day j, shift k -> 1 add to matrix; 0 otw
# S is work calendar of each staff, day is column & 5 is shift
S = np.full((N, D, 5), 0)

for staff in range(N):
    for day in range(D):
        for shift in range(1,5):
            S[staff, day, shift] = int(solver.Value(X[staff, day, shift])) # return {0;1}

# Label days & Shift to visualize
days = np.array([f"Day {day}" for day in range(1,D+1)])
shifts = np.array(["Morning", "Noon", "Afternoon", "Night"])

# Flat S by axis 0, use sum to return matrix include sum staff for each shift day
staff_per_shift = np.sum(S, axis=0)
# Create pandas DF to visualize
df_staff_shift = pd.DataFrame(data=staff_per_shift[:, 1:].T, index=shifts, columns=days)

# Visualize number staffs for each shift of day
df_staff_shift.style.background_gradient(cmap='Pastel2')

,Day 1,Day 2,Day 3,Day 4,Day 5
Morning,3,2,3,2,1
Noon,1,2,2,1,3
Afternoon,3,2,1,2,2
Night,2,1,2,1,1


In [32]:
col = np.array([f"Staff {staff}" for staff in range(1,N+1)])
row = days
details = np.full((D,N),"Rest")

for day in range(D):
  for staff in range(N):
    for shift in range(1,5):
      if S[staff, day, shift] == 1:
        details[day,staff] = shifts[shift-1]
        break

# Visualize details shift for each staff
pf_details = pd.DataFrame(data = details, index = row, columns = col)  
pf_details.style.applymap(status_color)

,Staff 1,Staff 2,Staff 3,Staff 4,Staff 5,Staff 6,Staff 7,Staff 8,Staff 9
Day 1,Morn,Noon,Afte,Morn,Morn,Afte,Nigh,Nigh,Afte
Day 2,Nigh,Noon,Morn,Morn,Afte,Afte,Rest,Rest,Noon
Day 3,Rest,Morn,Noon,Morn,Nigh,Nigh,Morn,Afte,Noon
Day 4,Afte,Rest,Morn,Nigh,Rest,Rest,Noon,Afte,Morn
Day 5,Noon,Nigh,Noon,Rest,Noon,Afte,Morn,Rest,Afte
